# Imports

In [22]:
import os
import pandas as pd


from dotenv import load_dotenv
import requests
from sqlalchemy import create_engine

from packages.utils import extract_player_salaries,load_into_db

In [18]:
dotenv_path = ("/Users/maukanmir/Documents/Machine-Learning/Web-Scraping-Code/Player-Salaries/dot.env")
load_dotenv(dotenv_path)

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
TABLE_NAME_PlAYER_SALARIES = "player_salaries"
TABLE_NAME_TEAM_SALARIES = "team_salaries"

engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}')

# Grabbing Player Salaries from 2011-2024

In [20]:
years = [str(year) for year in range(2011, 2025)]
all_data = pd.DataFrame()

for year in years:
  url = f"https://www.spotrac.com/nba/rankings/player/_/year/{year}/sort/cash_total"
  response = requests.get(url)
  year = int(year)
  df = extract_player_salaries(response.text, year)
  all_data = pd.concat([all_data, df])

# Download Dataset into DB

In [21]:
load_into_db(all_data, engine, TABLE_NAME_PlAYER_SALARIES)

Data successfully written to the database.


# Grabbing Player Salaries From 1989-2024

In [23]:
url = "https://hoopshype.com/salaries/players/2023-2024/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [34]:
tds = soup.findAll("td")

mega_string = " ".join([td.text.strip() for td in tds][1:])
cleaned_list = [block.strip().split(" ") for block in mega_string.split(".")[1:]]
df = []
for block in cleaned_list:
  players = {}
  for idx, player in enumerate(block):
    
    if "$" in player and "$" in player[idx-1]:
      continue
    elif "$" in player and player[idx-1].isalpha():
      players["salary"] = int(player.replace(",", "").replace("$", ""))
    if player.isalpha():
    

[['Stephen', 'Curry', '$51,915,615', '$51,915,615', '2'],
 ['Kevin', 'Durant', '$47,649,433', '$47,649,433', '3'],
 ['Nikola', 'Jokic', '$47,607,350', '$47,607,350', '3'],
 ['LeBron', 'James', '$47,607,350', '$47,607,350', '3'],
 ['Joel', 'Embiid', '$47,607,350', '$47,607,350', '6'],
 ['Bradley', 'Beal', '$46,741,590', '$46,741,590', '7'],
 ['Paul', 'George', '$45,640,084', '$45,640,084', '7'],
 ['Kawhi', 'Leonard', '$45,640,084', '$45,640,084', '7'],
 ['Giannis', 'Antetokounmpo', '$45,640,084', '$45,640,084', '7'],
 ['Damian', 'Lillard', '$45,640,084', '$45,640,084', '11'],
 ['Jimmy', 'Butler', '$45,183,960', '$45,183,960', '12'],
 ['Klay', 'Thompson', '$43,219,440', '$43,219,440', '13'],
 ['Rudy', 'Gobert', '$41,000,000', '$41,000,000', '14'],
 ['Fred', 'VanVleet', '$40,806,300', '$40,806,300', '15'],
 ['Anthony', 'Davis', '$40,600,080', '$40,600,080', '16'],
 ['Trae', 'Young', '$40,064,220', '$40,064,220', '16'],
 ['Zach', 'LaVine', '$40,064,220', '$40,064,220', '16'],
 ['Luka', 'Do